In [ ]:
!pip install -U kaleido

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 6.3 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import re
import pandas as pd
import numpy as np
from functools import reduce
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots


In [ ]:
path = '/content/drive/MyDrive/Cineuropa_Project/Model_Output/'

In [ ]:
df = pd.read_csv(path+'reviews.csv')

In [ ]:
cineuropa = pd.read_csv(path+'Cineuropa_reviews.csv', index_col='Unnamed: 0')
variety = pd.read_csv(path+'Variety_reviews.csv', index_col='Unnamed: 0')
hollywood = pd.read_csv(path+'Hollywoodreporter_reviews.csv', index_col='Unnamed: 0')
screen = pd.read_csv(path+'Screendaily_reviews.csv', index_col='Unnamed: 0')

In [ ]:
print(len(cineuropa))
print(len(variety))
print(len(hollywood))
print(len(screen))
print(101647+53741+51226+52727)

101647
53741
51226
52727
259341


In [ ]:
# Clean tensforflow probits
cineuropa['Probabilities_clean'] = cineuropa['Sentiment (Probabilities)'].apply(lambda x: re.findall(r'\d+\.\d+', x))
cineuropa['Probabilities_clean'] = cineuropa['Probabilities_clean'].apply(lambda x: [float(p) for p in x])

variety['Probabilities_clean'] = variety['Sentiment (Probabilities)'].apply(lambda x: re.findall(r'\d+\.\d+', x))
variety['Probabilities_clean'] = variety['Probabilities_clean'].apply(lambda x: [float(p) for p in x])

hollywood['Probabilities_clean'] = hollywood['Sentiment (Probabilities)'].apply(lambda x: re.findall(r'\d+\.\d+', x))
hollywood['Probabilities_clean'] = hollywood['Probabilities_clean'].apply(lambda x: [float(p) for p in x])

screen['Probabilities_clean'] = screen['Sentiment (Probabilities)'].apply(lambda x: re.findall(r'\d+\.\d+', x))
screen['Probabilities_clean'] = screen['Probabilities_clean'].apply(lambda x: [float(p) for p in x])

In [ ]:
def replace_na_with_nan(df):
    df[f'Predicted Sentiment'].loc[df['text'].isna()] = np.nan
    df[f'Sentiment (Probabilities)'].loc[df['text'].isna()] = np.nan

# Apply the function to each DataFrame
replace_na_with_nan(cineuropa)
replace_na_with_nan(variety)
replace_na_with_nan(hollywood)
replace_na_with_nan(screen)

<ipython-input-32-9355417425f8>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-32-9355417425f8>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-32-9355417425f8>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-32-9355417425f8>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pand

In [ ]:
cineuropa['Sentiment (Probabilities)']

0         tf.Tensor([0.00498053 0.99501944], shape=(2,),...
1         tf.Tensor([0.00106313 0.99893683], shape=(2,),...
2         tf.Tensor([0.00108127 0.9989188 ], shape=(2,),...
3         tf.Tensor([0.00108145 0.99891853], shape=(2,),...
4         tf.Tensor([0.00111744 0.9988826 ], shape=(2,),...
                                ...                        
101642    tf.Tensor([0.00142656 0.9985734 ], shape=(2,),...
101643    tf.Tensor([0.99381506 0.00618491], shape=(2,),...
101644    tf.Tensor([0.00704389 0.99295604], shape=(2,),...
101645    tf.Tensor([0.00146961 0.9985304 ], shape=(2,),...
101646    tf.Tensor([0.0012531 0.9987469], shape=(2,), d...
Name: Sentiment (Probabilities), Length: 101647, dtype: object

In [ ]:
def word_sep(data) :
    data['num_words'] = data['text'].apply(lambda x: len(x.split()) if isinstance(x, str) else np.nan)

word_sep(cineuropa)
word_sep(variety)
word_sep(hollywood)
word_sep(screen)


# Remove rows with too few words
cineuropa_new = cineuropa[cineuropa['num_words'] > 10]
variety_new = variety[variety['num_words'] > 10]
hollywood_new = hollywood[hollywood['num_words'] > 10]
screen_new = screen[screen['num_words'] > 10]

In [ ]:
def correct_probabilities(probabilities):
    corrected_probabilities = []

    for prob_list in probabilities:
        corrected_list = []
        for value in prob_list:
            if 9.9 < value < 10:
                corrected_list.append(round(value / 10, 7))
            elif 1 < value <= 9.8:
                corrected_list.append(round(value / 100, 7))
            else:
                corrected_list.append(value)
        corrected_probabilities.append(corrected_list)

    return corrected_probabilities

cineuropa_new['Probabilities_clean'] = correct_probabilities(cineuropa_new['Probabilities_clean'])
variety_new['Probabilities_clean'] = correct_probabilities(variety_new['Probabilities_clean'])
hollywood_new['Probabilities_clean'] = correct_probabilities(hollywood_new['Probabilities_clean'])
screen_new['Probabilities_clean'] = correct_probabilities(screen_new['Probabilities_clean'])

<ipython-input-36-9f60f39c648b>:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-36-9f60f39c648b>:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-36-9f60f39c648b>:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-36-9f60

In [ ]:
probs_screen = screen_new

# Extract probabilities for negative and positive sentiments
negative_probs = probs_screen[probs_screen['Predicted Sentiment'] == 0]['Probabilities_clean'].apply(lambda x: x[0])
positive_probs = probs_screen[probs_screen['Predicted Sentiment'] == 1]['Probabilities_clean'].apply(lambda x: x[1])

In [ ]:
# Create subplot
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Negative Sentiment Probabilities", "Positive Sentiment Probabilities")
)

# Add histograms
fig.add_trace(
    go.Histogram(
        x=negative_probs,
        nbinsx=100,
        opacity=0.75,
        name="Negative",
        marker_color=px.colors.sequential.Burgyl[2]
    ),
    row=1, col=1
)

fig.add_trace(
    go.Histogram(
        x=positive_probs,
        nbinsx=100,
        opacity=0.75,
        name="Positive",
        marker_color=px.colors.sequential.Burgyl[1]
    ),
    row=1, col=2
)

# Update y-axes and x-axes parameters for both subplots
fig.update_yaxes(range=[0, 200], showgrid=False, tickfont=dict(size=14), title=None, linewidth=1, linecolor='black', row=1, col=1)
fig.update_xaxes(range=[0.5, 1], showgrid=False, tickfont=dict(size=14), title_font=dict(size=16), linewidth=1, linecolor='black', row=1, col=1)

fig.update_yaxes(range=[0, 200], showgrid=False, tickfont=dict(size=14), title=None, linewidth=1, linecolor='black', row=1, col=2)
fig.update_xaxes(range=[0.5, 1], showgrid=False, tickfont=dict(size=14), title_font=dict(size=16), linewidth=1, linecolor='black', row=1, col=2)

# Update overall figure layout
fig.update_layout(
    title_text=None,
    showlegend=False,
    plot_bgcolor='white',
    width=700,  # Adjust total width for the subplot layout
    height=500
)

# Show the figure
fig.show()

In [ ]:
fig.write_image("/content/drive/MyDrive/Cineuropa_Project/Sent_probs_screen.pdf")

In [ ]:
def filter_rows(row):
    if row['Predicted Sentiment'] == 1:
        return np.max(row['Probabilities_clean']) > 0.98 if not np.isnan(row['Probabilities_clean']).all() else False
    elif row['Predicted Sentiment'] == 0:
        return np.max(row['Probabilities_clean']) > 0.80 if not np.isnan(row['Probabilities_clean']).all() else False
    else:
      return False

In [ ]:
filtered_cineuropa = cineuropa[cineuropa.apply(filter_rows, axis=1)]
filtered_variety = variety[variety.apply(filter_rows, axis=1)]
filtered_hollywood = hollywood[hollywood.apply(filter_rows, axis=1)]
filtered_screen = screen[screen.apply(filter_rows, axis=1)]

In [ ]:
mean_sent_cineuropa = filtered_cineuropa.groupby('ID')['Predicted Sentiment'].mean().reset_index()
mean_sent_cineuropa.rename(columns={'Predicted Sentiment': 'Predicted Sentiment Cineuropa'}, inplace=True)

mean_sent_variety = filtered_variety.groupby('ID')['Predicted Sentiment'].mean().reset_index()
mean_sent_variety.rename(columns={'Predicted Sentiment': 'Predicted Sentiment Variety'}, inplace=True)

mean_sent_hollywood = filtered_hollywood.groupby('ID')['Predicted Sentiment'].mean().reset_index()
mean_sent_hollywood.rename(columns={'Predicted Sentiment': 'Predicted Sentiment Hollywood Reporter'}, inplace=True)

mean_sent_screen = filtered_screen.groupby('ID')['Predicted Sentiment'].mean().reset_index()
mean_sent_screen.rename(columns={'Predicted Sentiment': 'Predicted Sentiment Screen Daily'}, inplace=True)


In [ ]:
print(mean_sent_cineuropa.describe())
print(mean_sent_variety.describe())
print(mean_sent_hollywood.describe())
print(mean_sent_screen.describe())

                ID  Predicted Sentiment Cineuropa
count  5760.000000                    5760.000000
mean   2880.500000                       0.759430
std    1662.913107                       0.156456
min       1.000000                       0.100000
25%    1440.750000                       0.666667
50%    2880.500000                       0.777778
75%    4320.250000                       0.875000
max    5760.000000                       1.000000
                ID  Predicted Sentiment Variety
count  2082.000000                  2082.000000
mean   3503.293948                     0.590329
std    1654.661367                     0.189721
min       5.000000                     0.000000
25%    2158.250000                     0.462225
50%    3823.500000                     0.608696
75%    4987.750000                     0.733333
max    5759.000000                     1.000000
                ID  Predicted Sentiment Hollywood Reporter
count  1837.000000                             1837.000000


In [ ]:
dfs = [df, mean_sent_cineuropa, mean_sent_variety, mean_sent_screen]
result_df = reduce(lambda left, right: pd.merge(left, right, on='ID'), dfs)

In [ ]:
merged_1 = pd.merge(df, mean_sent_cineuropa, on='ID', how='left')
merged_2 = pd.merge(merged_1, mean_sent_variety, on='ID', how='left')
merged_3 = pd.merge(merged_2, mean_sent_hollywood, on='ID', how='left')
merged_4 = pd.merge(merged_3, mean_sent_screen, on='ID', how='left')

In [ ]:
merged_4[merged_4['original_title']== "Monica"]

,ID,url,title,original_title,director,year,country,cineuropa_review_author,cineuropa_review_text,cineuropa_review_date,...,hollywoodreporter_review_date,screendaily_review_author,screendaily_review_text,screendaily_review_date,rottentomatoes_tomatometer_score,rottentomatoes_audience_score,Predicted Sentiment Cineuropa,Predicted Sentiment Variety,Predicted Sentiment Hollywood Reporter,Predicted Sentiment Screen Daily
102,103,https://www.cineuropa.org/en/newsdetail/429818/,Monica,Monica,Andrea Pallaoro,2022,"United States, Italy",Andrea Pallaoro,Andrea Pallaoro is back competing in the Venic...,04/09/2022,...,03/09/2022,NaN,\n\nSource: Required Viewing\n\n‘Monica’\n\nDi...,03/09/2022,NaN,NaN,0.333333,0.444444,0.351351,0.761905


In [ ]:
merged_4.to_csv(path+'MasterSentiment03012023.csv')